In [46]:
import pandas as pd
import numpy as np
from datetime import datetime

import os

from icalendar import Calendar, Event
import pytz

In [3]:
try:
    os.mkdir('data')
except FileExistsError:
    print('data directory already exists')

In [51]:
data = pd.read_csv('data/timetable.csv')

In [35]:
data.head()

,date,room,time,group,class,session,tutor
0,2025-03-10,LI6,09:00-10:30,1i,Conference,2.0,MS/KA
1,2025-03-10,LI6,10:45-12:15,1i,Conference,2.0,MS/KA
2,2025-03-10,LI6,12:45-14:15,1ii,Conference,2.0,MS
3,2025-03-10,LI6,14:30-16:00,1ii,Conference,2.0,MS
4,2025-03-10,LI7,09:00-10:30,2i,Conference,2.0,SS


In [36]:
data['date'] = pd.to_datetime(data['date'])

In [52]:
toby_sched = data[data['group'].isin(['9', '9A', '9ii'])].sort_values(by=['date', 'time'])

In [53]:
def csv_to_ics(csv_input, output_file="timetable.ics"):
    """
    Convert a CSV timetable to an ICS (iCalendar) file.
    
    Parameters:
    - csv_input: Path to CSV file (str) or pandas DataFrame
    - output_file: Path to save the ICS file (default: "timetable.ics")
    """
    # Load CSV into DataFrame if a file path is provided
    if isinstance(csv_input, str):
        df = pd.read_csv(csv_input)
    elif isinstance(csv_input, pd.DataFrame):
        df = csv_input
    else:
        raise ValueError("csv_input must be a file path (str) or pandas DataFrame")

    # Initialize the calendar
    cal = Calendar()
    cal.add('prodid', 'Max Roulstone')
    cal.add('version', '2.0')

    # Set timezone (assuming UTC; adjust if needed)
    timezone = pytz.UTC

    # Process each row in the DataFrame
    for _, row in df.iterrows():
        # Parse date and time
        event_date = datetime.strptime(row['date'], '%Y-%m-%d')
        start_time, end_time = row['time'].split('-')
        start_dt = timezone.localize(datetime.strptime(f"{row['date']} {start_time}", '%Y-%m-%d %H:%M'))
        end_dt = timezone.localize(datetime.strptime(f"{row['date']} {end_time}", '%Y-%m-%d %H:%M'))

        # Create an event
        event = Event()
        event.add('dtstart', start_dt)
        event.add('dtend', end_dt)
        event.add('summary', f"{row['class']} Session {row['session']}")
        event.add('location', row['room'])
        event.add('description', f"Group: {row['room']}, Tutor: {row['tutor']}")
        event.add('uid', f"{start_dt.strftime('%Y%m%dT%H%M%S')}-{row['room']}@mr")  # Unique ID

        # Add event to calendar
        cal.add_component(event)

    # Write to ICS file
    with open(output_file, 'wb') as f:
        f.write(cal.to_ical())
    print(f"ICS file saved as {output_file}")

In [54]:
csv_to_ics(toby_sched, 'data/toby_schedule.ics')

ICS file saved as data/toby_schedule.ics
